In [2]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import torch
import re
from bs4 import BeautifulSoup
from io import StringIO
import csv 

df = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')
table_descriptions = "The csv contains financial data from an S&P 500 earnings."

def convert_to_csv_in_stringio_format(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    # Write rows to the StringIO object
    writer = csv.writer(csv_buffer)
    trows = table.find_all('tr')
    for trow in trows:
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    csv_data = csv_buffer.getvalue()
    return csv_data

def convert_to_csv(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    # Write rows to the StringIO object
    writer = csv.writer(csv_buffer)
    trows = table.find_all('tr')
    final_csv = ''
    for trow in trows:
        writer = csv.writer(csv_buffer)
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
        csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
        csv_buffer = StringIO()
        final_csv += csv_data + '\n'
    return final_csv

df['csv_content'] = df['html_content'].apply(convert_to_csv)
def create_prompt(row):
    csv_content = row['csv_content']
    question = row['question']
    return (
        f"""[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].
        ### [Table Text]
        {table_descriptions}

        ### [Table]
        ```
        {csv_content}
        ```

        ### [Question]
        {question}

        ### [Solution][/INST]"""
    )

df["value"] = df.apply(create_prompt, axis=1)
df = df.dropna()
print(df.columns)
custom_ds = pd.DataFrame()
custom_ds["prompt"] = df['value']

train_dataset = Dataset.from_pandas(custom_ds)

/home/ubuntu/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['answer', 'html_content', 'index', 'question', 'pseudo_json_encoding',
       'csv_content', 'value'],
      dtype='object')


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "/home/ubuntu/Large-Language-Model-for-Key-Value-Extractions/notebooks/results_V12/checkpoint-160"
config = PeftConfig.from_pretrained(peft_model_id)


In [4]:
# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  device_map={"":0}, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.85s/it]


Peft model loaded


In [5]:
print("Test sample")
text = train_dataset["prompt"][10]
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
# outputs = model.generate(**inputs, max_new_tokens=50,return_dict_in_generate=True, output_scores=True)
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=2048,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Test sample
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].
        ### [Table Text]
        The csv contains financial data from an S&P 500 earnings.

        ### [Table]
        ```
        " 1"," -"," global payments"," s&p 500"," s&p information technology"
" 2"," may 31 2002"," $ 100.00"," $ 100.00"," $ 100.00"
" 3"," may 31 2003"," 94.20"," 91.94"," 94.48"
" 4"," may 31 2004"," 129.77"," 108.79"," 115.24"
" 5"," may 31 2005"," 193.30"," 117.75"," 116.29"
" 6"," may 31 2006"," 260.35"," 127.92"," 117.14"
" 7"," may 31 2007"," 224.24"," 157.08"," 144.11"

        ```

        ### [Question]
        What is may 31 2005 for global payments ?

        ### [Solution][/INST] 193.3


In [6]:
# from random import randrange
# sample = dataset['test'][randrange(len(dataset["test"]))]
text = train_dataset["prompt"][16000]
input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=30, do_sample=True, top_p=0.9)
print(f"input sentence: {text}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input sentence: [INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].
        ### [Table Text]
        The csv contains financial data from an S&P 500 earnings.

        ### [Table]
        ```
        " 1"," 2008"," $ 317"
" 2"," 2009"," 275"
" 3"," 2010"," 236"
" 4"," 2011"," 214"
" 5"," 2012"," 191"
" 6"," later years"," 597"
" 7"," total minimum payments required"," $ 1830"

        ```

        ### [Question]
        What is 2008 ?

        ### [Solution][/INST]
------------------------------------------------------------
summary:
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].
        ### [Table Text]
        The csv contains financial data from an S&P 500 earnings.

        ### [Table]
        ```
        " 1"," 2008"," $ 317"
" 2"," 2009"," 275"
" 3"," 2010"," 236"
" 4"," 2011"," 214"
" 5"," 2012"," 191"
" 6"," later years"," 597"
" 7"," total minimum payments r

In [7]:
def extract_inst(output):
    # Find the index of [/INST]
    index = output.find("[/INST]")

    # Extract the value after [/INST]
    value_after_inst = output[index + len("[/INST]"):].strip()
    return value_after_inst 

In [8]:
answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [9]:
extract_inst(answer)

'$317'

In [10]:
output_df = pd.DataFrame(columns=['index', 'csv_content', 'question', 'answer', 'prediction'])

for i in tqdm(range(15000, len(train_dataset))):
    try:
        text = train_dataset["prompt"][i]
        input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()
        # with torch.inference_mode():
        outputs = model.generate(input_ids=input_ids, max_new_tokens=30, do_sample=True, top_p=0.9)
        decoded_output = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
        
        inst_values = extract_inst(decoded_output)
        # print(decoded_output)
        # print(inst_values)
        # # print(df.loc[i,'csv_content'].replace('  ', ''))
        # print(df.loc[i,'question'])
        # print(df.loc[i,'answer'])
        print('***************')
        output_df.at[i, 'index'] = df.loc[i, 'index']
        output_df.at[i, 'question'] = df.loc[i,'question']
        output_df.at[i, 'answer'] = df.loc[i,'answer']
        output_df.at[i, 'csv_content'] = df.loc[i,'csv_content'].replace('  ', '')
        output_df.at[i, 'prediction'] = inst_values
    except:
        print(f'error')

output_df.to_csv('results_V12/eval.csv', index=False)



  0%|          | 0/2525 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/2525 [00:00<29:59,  1.40it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***************


  0%|          | 2/2525 [00:01<22:06,  1.90it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***************


  0%|          | 3/2525 [00:01<25:46,  1.63it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

***************


 96%|█████████▋| 2431/2525 [29:34<00:58,  1.60it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***************


 96%|█████████▋| 2432/2525 [29:34<00:50,  1.86it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


***************
